# Introduction

## Iteration 1
- **Objective:** The primary focus is to expand the dataset with new financial data from asr, improve model performance and incorporate feedback
- **Outcome**
    - **Data Enhancement:** Added the new financial data from asr
    - **Model Evolution:** Switched to time series rolling cross validation
    - **Parameter Optimization:** New parameters where selected due to addition of new financial data from asr
    - **General:** Model improved MSE and MAE, also the model become more transparant and is better optimized for the small dataset

## Previous iteration information
### Iteration zero
**Iteration Zero**
- **Objective:** Find out if the project is viable
- **Outcome:** The data analysis in phase 2 faced challenges due to financial calculation changes from 2023 and restricted access to historical analyst consensus data, limiting our dataset's effectiveness. The baseline model underperformed, likely due to insufficient target variable optimization and missing key predictive features. Future iterations should focus on overcoming data limitations, refining target optimization, and expanding the dataset to enhance model performance and mitigate biases.

## Importing libraries

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

print("pandas version: ", pd.__version__)
print("seaborn version:", sns.__version__)
print('numpy version:', np.__version__)

pd.set_option('display.float_format', '{:.2f}'.format)

# Phase 2

## Data Requirements
This data dictionary is for all data that will be used for our model. We have taken this out of the data requirement document (this can be found in this folder under the name data_requirements.docx), A changelog will be kept in the data requirement document.

### Data Dictionary
| Dutch Term              | English Term             | Description                                                        | Data Type      | Range                        | Units          | Source          | Quality Standards          |
|-------------------------|--------------------------|--------------------------------------------------------------------|----------------|------------------------------|----------------|-----------------|----------------------------|
| Datum                   | Date                     | The date when the analyst ratings were recorded.                   | Date           | 2022-07-14 to 2022-12-01     | Date           | Marketscreener     | Accurate date required     |
| Kopen                   | Buy                      | Number of 'Buy' ratings from analysts.                             | Numerical      | 0 - 15                       | Ratings        | Marketscreener     | Accurate count             |
| Overpresteren           | Outperform               | Number of 'Outperform' ratings from analysts.                      | Numerical      | 0 - 15                       | Ratings        | Marketscreener     | Accurate count             |
| Houden                  | Hold                     | Number of 'Hold' ratings from analysts.                            | Numerical      | 0 - 15                       | Ratings        | Marketscreener     | Accurate count             |
| Onderpresteren          | Underperform             | Number of 'Underperform' ratings from analysts.                    | Numerical      | 0 - 15                            | Ratings        | Marketscreener     | Accurate count             |
| Verkopen                | Sell                     | Number of 'Sell' ratings from analysts.                            | Numerical      | 0 - 15                            | Ratings        | Marketscreener     | Accurate count             |
| Datum | Date         | The date when the stock market data was recorded.                           | Date       | Varied (min 11-06-2016 etc.)  | Date     | Yahoo Finance| Accurate date required   |
| Open | Open         | Opening price of the stock for the given date.                              | Numerical  | Varied (min 0,.-max ∞)                    | Currency | Yahoo Finance| Accurate financial data  |
| Hoog | High         | Highest price of the stock reached on the given date.                       | Numerical  | Varied (min 0,.-max ∞)                   | Currency | Yahoo Finance| Accurate financial data  |
| Laag | Low          | Lowest price of the stock on the given date.                                | Numerical  | Varied (min 0,.-max ∞)                   | Currency | Yahoo Finance| Accurate financial data  |
| Slot | Close        | Closing price of the stock at the end of the trading day on the given date. | Numerical  | Varied (min 0,.-max ∞)                   | Currency | Yahoo Finance| Accurate financial data  |
| Aangpaste slot | Adj Close    | Adjusted closing price after adjustments for all applicable splits and dividend distributions.| Numerical | Varied (min 0,.-max ∞) | Currency | Yahoo Finance| Accurate financial data  |
| Volume | Volume       | Number of shares of the stock traded during the given date.                 | Numerical  | Varied (min 0,.-max ∞)                   | Shares   | Yahoo Finance| Accurate count           
| Datum       | Date    | The date from the financial report.    | Date   | 2022-01-01 to 2023-01-01  | Date   | [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)   | Accurate date required     
| Eigen Vermogen Rendement (%)| Return On Equity (%)| income level a firm is generating as a percentage of shareholder's equity| Numerical| 11.3 - 15.3 | Percent | [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)  | Accurate percentage        
| Andere uitgebreide inkomsten | Other Comprehensive Income           | Total comprehensive income not reported in the profit or loss.     | Numerical    | -874000000 - 562000000        | Currency       | [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)   | Accurate financial data    
| Gecombineerde Ratio (%)  P&C and Arbeidsongeschiktheid   | Combined Ratio P&C and Disability (%)| The combined ratio for property and casualty, and disability insurance sectors. | Numerical    | 91.7 - 92.8                  | Percent        | [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)   | Accurate percentage        
| Uitstaande Aandelen (gewogen gemiddelde)| Outstanding Shares (Weighted Average) | Weighted average of outstanding shares during the period.         | Numerical    | 134999182.84 - 137004579.72  | Shares         | [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)   | Accurate count 
| Publicatie                      | Publication                          | The financial period the data represents    | Text            | e.g., FY2021, HY2022, FY2022 | N/A            | [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)   | Accurate financial period    |
| Bestandsnaam1                   | Filename1                            | The name of the first file from ASR where the data was sourced.    | Text            | Varied                       | N/A            | [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)   | Accurate file reference      |
| Bestandsnaam2                   | Filename2                            | The name of the second file from ASR where the data was sourced.   | Text            | Varied                       | N/A            | [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)   | Accurate file reference      |

## Data Collection

### Overview

- **Analyst consensus**: Extracted by typing over the data from an interactive graph, Currently there are no free sources to get this data.
- **Financial data from ASR**: Looked through the financials from ASR on their website, then extracted the neccessary data by typing it over in a new excel file. This data is collected pre-IFRS 17/9
- **Stock data for ASR**: On the Yahoo website there is a page to download the data by Day, Month or Year. The data range can be stated before downloading.

### Detailed process
We will explain exactly how we collected the data so these steps can be replicated.

#### Analyst consensus
- **Source**: [marketscreener.com](https://www.marketscreener.com/quote/stock/ASR-NEDERLAND-N-V-28377340/consensus/)
- **Limitation**: Not downloadable data and only up to 18 months of history.
- **Time Frame**: Data from 2022 July up to 2023 Januari. With a moving time frame(Only 18 months back from current date). With around 2 weeks between every entry
- **Script**: No script used, we typed over the values from the website in to an excel file.
- **Storage**: Data stored locally in the data folder. Filename: [analyst_consensus_16_07-2022_17-01-2024.xlsx](data/analyst_consensus_16_07-2022_17-01-2024.xlsx).
- **Future Data Addition**: Look for a source with better historical data that is easier to gather


#### Financial data from asr
- **Source**: [asrnederland.nl](https://www.asrnederland.nl/investor-relations/financiele-publicaties)
- **Limitation**: The report are per half year or full year. The best would be that a shorter time frame is available. This is not likely to happen 
- **Time Frame**: Financial data available from 2016 up to 2023. (asr whent public in 2016 June). Data collected from the following publications: FY2020, HY2021, FY2021, HY2022, FY2022
- **Script**: No script used. Downloaded the 'Tables' and 'Financial ratios' under column 'Halfjaarcijfers' in the tab from '2022' and '2023'.
- **Storage**: Data stored locally in the data folder. Filename: [extracted_financial_data_Pre-IFRS_FY2021_HY2022_FY2022.xlsx](data/extracted_financial_data_Pre-IFRS_FY2021_HY2022_FY2022.xlsx)
The original data from the asr website is stored in the folder named 'Original data asr'
- **Future Data Addition**: When newer reports are released these should be downloaded and the data should be added to [extracted_financial_data_Pre-IFRS_FY2021_HY2022_FY2022.xlsx](data/extracted_financial_data_Pre-IFRS_FY2021_HY2022_FY2022.xlsx)

---
- **Important note**: We excluded the IFRS 17/9 financial data during collection. This is first recorderd from the publication HY2023. They also have recalculated metrics for HY2022 and FY2022. This data is not useful for our project at this moment in time
---

#### Stock data for asr
- **Source**: [finance.yahoo.com](https://finance.yahoo.com/quote/ASRNL.AS/history)
- **Limitation**: 
- **Time Frame**: Financial data available from 2016 up to 2023. (asr whent public in 2016 July)
- **Script**: No script used. Under the tab history there is the option to download the data from a given date as daily, monthly or yearly data. The download is in a csv format.
- **Storage**: Data stored locally in the data folder. We downloaded the full data available in three files
  - Daily: Filename: [StockPerDay_ASRNL.AS_01-01-2016_19-01-2024.csv](data/StockPerDay_ASRNL.AS_01-01-2016_19-01-2024.csv)
  - Weekly: Filename: [StockPerWeek_ASRNL.AS_01-01-2016_19-01-2024.csv](data/StockPerWeek_ASRNL.AS_01-01-2016_19-01-2024.csv)
  - Yearly: Filename: [StockPerYear_ASRNL.AS_01-01-2016_19-01-2024.csv](data/StockPerYear_ASRNL.AS_01-01-2016_19-01-2024.csv)
- **Future Data Addition**: When newer data is available for `Analyst consensus` or `Financial data from ASR` then from the yahoo source the new stock data should be downloaded and replace the existing files for daily, monthly and yearly

### Data Handling
All data handling, including preprocessing and cleaning, will be conducted within this notebook, focusing on highways in the Netherlands.

## Data understanding & Preparation for analyst consensus
In this part we will look at the data for the analyst consensus. This will be the most important data for our target variable.

We will start with loading the data and taking a first look

In [ ]:
AnalystConsensus = pd.read_excel('data/analyst_consensus_16_07-2022_17-01-2024.xlsx')
AnalystConsensus.head(10)

The data is loaded correctly. We can see from this;
- one column to indicate the date
- 5 columns to show the count of the analyst consensus
- There seems to be around two weeks between every entry

We will now describe the dataset

In [ ]:
AnalystConsensus.describe()

We now know there are 37 entries into the dataset.
The newest datapoint is from 17-01-2024 and the oldest is from 14-07-2022.

What is also visible is that the max count of the consensus fields is in all cases below 9. From this we can see that at maximum there are not a lot of analyst giving their opinion about asr

Now we will look if the types are correct with the .info

In [ ]:
AnalystConsensus.info()

Here we see that everything has the correct datatype and there are no null values

### Data visualization
Below we will create an interactive plot to visualize the data in our dataset

In [ ]:
df = AnalystConsensus

# Initialize go
fig = go.Figure()

# add bars
fig.add_trace(go.Bar(
    x=df['Date'],
    y=df['Sell'],
    name='Sell',
    marker_color='red'
))
fig.add_trace(go.Bar(
    x=df['Date'],
    y=df['Underperform'],
    name='Underperform',
    marker_color='orange'
))
fig.add_trace(go.Bar(
    x=df['Date'],
    y=df['Hold'],
    name='Hold',
    marker_color='yellow'
))
fig.add_trace(go.Bar(
    x=df['Date'],
    y=df['Outperform'],
    name='Outperform',
    marker_color='lightgreen'
))
fig.add_trace(go.Bar(
    x=df['Date'],
    y=df['Buy'],
    name='Buy',
    marker_color='green'
))

# update the figure
fig.update_layout(
    barmode='stack',
    title='Analyst Consensus - Interactive Stacked Bar Chart',
    xaxis_title='Date',
    yaxis_title='Consensus Count',
    legend_title='Consensus',
    hovermode='x'
)

fig.show()

From this plot we can see that analust overall think that asr is a good buy. With a slight hiccup around march where there was a sell consensus.

We can also see that there is a difference in the count over the weeks, this is something to look into in the prepocessing step of Phase 3 as this can influence the machine learning.


### Conclusion
The data for the analyst consensus did not need any altering. The data is clear and we have a good visualization that shows the consensus over time, what we did notice it the varying count over the weeks. This is something we need to look at in the preprocessing step.

#### Next step
Find out what happened in and around march that negatively infleanced the analyst consensus to ``sell``

## Data understanding & Preparation for Financial data from asr
In this section we will look at the financial data gathered from asr.

We will start with loading the data and taking a first look

In [ ]:
FinancialDataASR = pd.read_excel('data/extracted_financial_data_Pre-IFRS_FY2021_HY2022_FY2022.xlsx')
FinancialDataASR.head()

We can see that for the date `2021-01-01` and `2021-06-30` not all entries are filled in. 
Due to the way we collected the data we know that the OCI for publication `HY2021` was not available in the report from asr. For the publication `FY2020` we explicitly left fields open due to time constraint.

In this iteration we will choose to remove the first two fields. We will do this below


In [8]:
FinancialDataASR = FinancialDataASR[~FinancialDataASR['publication'].isin(['FY2020', 'HY2021'])]

Now that the code executed we will use an assert to check if the values are succesfully removed

In [9]:
assert not FinancialDataASR['publication'].isin(['FY2020', 'HY2021']).any()

No error is thrown. We will reset the index and check if the datatypes are correct

In [ ]:
FinancialDataASR.reset_index(drop=True)

In [ ]:
FinancialDataASR.info()

The datatypes are correct. Since the dataset is so small at this point we will not use the .describe. All information is visible in the .head() and with that visual and our knowledge about the data collection, no strange values are seen. 

### Data visualization
Since the dataframe only contains 3 rows we will not create any visualization as this does not add anything to our understanding. From looking at the dataframe we can get a good sense of what is in the data and the connection between the data

## Data understanding & Preparation for Stock data for asr
In this section we will look at the stock data gathered for asr.

The analyst consensus data has a new entry around every two weeks. We want to have our data matching to this timeframe. For the stock data we will use the data gathered per week.
We will start with loading the data and taking a first look.

In [ ]:
StockPerWeek = pd.read_csv('data/StockPerWeek_ASRNL.AS_01-01-2016_19-01-2024.csv', delimiter=',', decimal='.')
StockPerWeek.head(10)

We can see that this dataframe columns for the date. We can see that the data is indeed weekly since every 7 days there is a new entry.

The dataframe also contains data for the Open, High, Low and Volume of that week.

The *Close and **Adj Close are fields with a description. This is the explanation from Yahoo;
- `*Close price adjusted for splits.`
- `**Adjusted close price adjusted for splits and dividend and/or capital gain distributions.`

In the next step we will use .info

In [ ]:
StockPerWeek.info()

We can see that there are 397 entries for all columns and we can also see that there are no null values.

for the datatypes we need to change the data from object to a date, we will do this in the code block below

In [14]:
# Convert 'Date' to datetime
StockPerWeek['Date'] = pd.to_datetime(StockPerWeek['Date'])

In [ ]:
StockPerWeek.info()

From the .info above we can see that the conversion of `Date` was succesful.
Now we will use .describe to look for anomalies

In [ ]:
StockPerWeek.describe()

The .describe shows that the dataset appears normal, with no extreme values observed in any of the columns. One observation is that the earliest date in the dataset is June 13, 2016. This date represents more historical data than is currently required for matching with the analyst consensus.

Going forward, our plan includes the integration of additional analyst consensus data. This means that we will review this entire dataset, starting from 2016. This ensures that we can effectively incorporate new consensus data as it becomes available.

### Data visualization
Below we will start the visualization with creating a plot to visualize the volume over time

In [ ]:
import matplotlib.ticker as ticker

df = StockPerWeek

plt.figure(figsize=(10, 6))
plt.bar(df['Date'], df['Volume'], color='blue', label='Trading Volume', width=3)
plt.title('Trading Volume Over Time')
plt.xlabel('Date')
plt.ylabel('Volume')

# This is needed to transform the mathemetical notation to numbers
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))

# Set the limit to 8.000.000 to capture most datapoint and still have it readable
plt.ylim(0, 8000000)

plt.legend()
plt.show()

Based on the provided plot, which appears to show the trading volume over time, we can observe that there is a fluctuating but somewhat consistent pattern of trade volume throughout the period from 2017 to 2024. There are notable spikes in volume at certain intervals, which could correspond to specific market events, earnings announcements, or other news affecting trading activity. However, the majority of trading days seem to have a volume well below these peaks. The consistency of the lower volume suggests a baseline level of trading activity, while the spikes indicate periods of high trader interest or market volatility. It would be interesting to cross-reference the spikes with actual market events to draw more detailed conclusions on what may have caused these surges in trading volume.

We also want to see the close price over time as this gives us a good overview of the stock movement.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['Close'], label='Close Price')
plt.title('Closing Stock Prices Over Time')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

It would be nice to see the two plots overlapping with each other to see if there is an effect on the close price due to trading volume.

In [ ]:
plt.figure(figsize=(10, 6))

# Create the first plot for trading volume on the primary y-axis
ax1 = plt.gca()  # Get the current Axes instance on the current figure
ax1.bar(df['Date'], df['Volume'], color='blue', label='Trading Volume', width=3)
ax1.set_xlabel('Date')
ax1.set_ylabel('Volume', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
ax1.set_ylim(0, 8000000)  # Set the limit to capture most datapoints for volume
ax1.legend(loc='upper left')

# Create a secondary y-axis for the closing price
ax2 = ax1.twinx()  # Instantiate a second axes that shares the same x-axis
ax2.plot(df['Date'], df['Close'], label='Close Price', color='green')
ax2.set_ylabel('Price', color='green')
ax2.tick_params(axis='y', labelcolor='green')
ax2.legend(loc='upper right')

plt.title('Trading Volume and Closing Stock Prices Over Time')
ax1.grid(True)
plt.show()

It does not appear that there is a direct correlation between the stock price movement and trading volume.

It is noteworthy that there are a few occasions where the stock price drops dramatically and then we can also see a massive spike in trading volume.

## Concatenating data from Stock market and Analyst consensus
We know from our domain understanding that the consensus is often reflected in the stock market or the other way around.

In this section we will add the datasets together, we will lose a part of the stock market data by doing this.

First we will add a column to both dataset to state the number of the week and one column for the year, afterards we will join them based on the week number.
We will start with the analyst consensus dataframe

In [ ]:
# AnalystConsensus

# Creating a column named Week to store the week number
AnalystConsensus['Week'] = AnalystConsensus['Date'].dt.isocalendar().week

# Creating a column named Year to store the year number
AnalystConsensus['Year'] = AnalystConsensus['Date'].dt.year

# Display the first rows to verify
AnalystConsensus.head(3)

Now the ``AnaylstConsensus`` is done we will do the same for the ``StockPerWeek`` dataframe

In [ ]:
# StockPerWeek

# Creating a column named Week to store the week number
StockPerWeek['Week'] = StockPerWeek['Date'].dt.isocalendar().week

# Creating a column named Year to store the year number
StockPerWeek['Year'] = StockPerWeek['Date'].dt.year

# Display the first rows to verify
StockPerWeek.head(3)

Now that we added the week column we need to edit the stock market dataframe.

First we will look at the max and min week + year of the analyst consensus dataframe

In [ ]:
# Find the minimum and maximum years in the DataFrame
min_year = AnalystConsensus['Year'].min()
max_year = AnalystConsensus['Year'].max()

# Filtering AnalystConsensus to only include rows from the min year
min_year_df = AnalystConsensus[AnalystConsensus['Year'] == min_year]

# Now we look for the min week within the min year
min_week_in_min_year = min_year_df['Week'].min()

# Here we will do the same for max
max_year_df = AnalystConsensus[AnalystConsensus['Year'] == max_year]
max_week_in_max_year = max_year_df['Week'].max()


print(f"The minimum week in the minimum year {min_year} is: {min_week_in_min_year}")
print(f"The maximum week in the maximum year {max_year} is: {max_week_in_max_year}")

Now that we know the min is week 28 of the year 2022 and the max is week 3 of the year 2024.

We will remove all fields that do not fall inbetween these weeks from the stock market data

In [ ]:
# Filter the DataFrame to include rows that fall within the given range
# Include all weeks in the min_year after min_week_in_min_year
# Include all weeks in the max_year up to and including max_week_in_max_year
# For years between min_year and max_year, include all weeks

filtered_df = StockPerWeek[
    ((StockPerWeek['Year'] == min_year) & (StockPerWeek['Week'] >= min_week_in_min_year)) |
    ((StockPerWeek['Year'] == max_year) & (StockPerWeek['Week'] <= max_week_in_max_year)) |
    ((StockPerWeek['Year'] > min_year) & (StockPerWeek['Year'] < max_year))
]

# Now let's print the shape of the original and filtered DataFrames to see how many rows were removed
print(f"Original DataFrame shape: {StockPerWeek.shape}")
print(f"Filtered DataFrame shape: {filtered_df.shape}")

According to the shape we can see that changes have been made. There now should be 80 rows in the dataset. This appears to be the correct time range. We will print the max and min week + year to check if the values are the same as in the ``AnalystConsensus`` dataset

In [ ]:
StockPerWeek = filtered_df

# Find the minimum and maximum years in the DataFrame
min_year = StockPerWeek['Year'].min()
max_year = StockPerWeek['Year'].max()

# Filtering AnalystConsensus to only include rows from the min year
min_year_df = StockPerWeek[StockPerWeek['Year'] == min_year]

# Now we look for the min week within the min year
min_week_in_min_year = min_year_df['Week'].min()

# Here we will do the same for max
max_year_df = StockPerWeek[StockPerWeek['Year'] == max_year]
max_week_in_max_year = max_year_df['Week'].max()


print(f"The minimum week in the minimum year {min_year} is: {min_week_in_min_year}")
print(f"The maximum week in the maximum year {max_year} is: {max_week_in_max_year}")

Now we know that the min and max values we will perform one last check. See if the weeknumbers + the year are unique in each datasets. This way we can check if any data was incorrectly changed during the transformation steps above, **as an example; one weeknumber twice in the dataset is wrong**

In [ ]:
# Create new DataFrames
df1 = pd.DataFrame(StockPerWeek)
df2 = pd.DataFrame(AnalystConsensus)

# Combine 'Week' and 'Year' into a single string for each DataFrame
df1['WeekYear'] = df1['Week'].astype(str) + df1['Year'].astype(str)
df2['WeekYear'] = df2['Week'].astype(str) + df2['Year'].astype(str)

# Convert to lists
list1 = df1['WeekYear'].tolist()
list2 = df2['WeekYear'].tolist()

# Use assert to check if each DataFrame's 'WeekYear' column has only unique values
try:
    # check all number of unique values againts the numbers of items in de dataframe
    assert df1['WeekYear'].nunique() == len(df1['WeekYear'])
    assert df2['WeekYear'].nunique() == len(df2['WeekYear'])
    print("Both DataFrames have only unique 'WeekYear' values.")
except AssertionError:
    print("One or both DataFrames do not have only unique 'WeekYear' values.")

From this we can see that they are the same, now we can start joining the two dataset.

In [ ]:
# Merging the two DataFrames on 'Year' and 'Week' using an inner join.
merged_df = pd.merge(StockPerWeek, AnalystConsensus, on=['Year', 'Week'], how='inner')

# This will result in a DataFrame that only contains rows where both 'Year' and 'Week' match in both DataFrames.
# All weeks that do not exist in AnalystConsensus will be dropped.

# Displaying the shape of the new df
print(f"Merged DataFrame shape: {merged_df.shape}")


merged_df.head()

Now that the two dataframes are merged on week and year we will continue to Phase 3.

Now we have added the ``AnalystConsensus`` and `StockPerWeek` will add the last dataframe `FinancialDataASR` to the merged dataset and use linear interpolation to fill in the gaps. We will first look at the dataframe

In [ ]:
FinancialDataASR.head()

We will handle this the same as the previous dataframes. We will start by adding a column for ``Week`` and ``Year``

In [ ]:
# FinancialDataASR

# Creating a column named Week to store the week number
FinancialDataASR['Week'] = FinancialDataASR['date'].dt.isocalendar().week

# Creating a column named Year to store the year number
FinancialDataASR['Year'] = FinancialDataASR['date'].dt.year

# Check the transformation
FinancialDataASR.head()

We see that the code, because of isocalendat, made the first of januari week 52. This is not exactly what we want so we will tranform this value 52 to 1

In [ ]:
df = FinancialDataASR

# Create a month column
df['Month'] = df['date'].dt.month

# Find rows where 'Week' is 52 and 'Month' is January (1)
condition = (df['Week'] == 52) & (df['Month'] == 1)

# Update the 'Week' column for these rows to 1
df.loc[condition, 'Week'] = 1

df.head()

This was succesful, now we will drop the ``Month`` column

In [ ]:
df = df.drop(columns=['Month'])
df.head()

We will now use linear interpolation to fill in the weeks between the minimum and maximum date

In [ ]:
df.info()

We will now fill the dataframe with empty rows for the weeks that will be filled in

In [ ]:
# Adjust the week number for specific dates
df['Week'] = df['date'].dt.isocalendar().week
df['Year'] = df['date'].dt.year
df.loc[(df['date'].dt.month == 1) & (df['Week'] > 50), 'Week'] = 1

# Determine the range of dates
min_date = df['date'].min()
max_date = df['date'].max()

# Generate a complete range of dates
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
df_all_dates = pd.DataFrame(all_dates, columns=['date'])
df_all_dates['Week'] = df_all_dates['date'].dt.isocalendar().week
df_all_dates['Year'] = df_all_dates['date'].dt.year

# Adjust the week number for the all_dates DataFrame as well
df_all_dates.loc[(df_all_dates['date'].dt.month == 1) & (df_all_dates['Week'] > 50), 'Week'] = 1

# Merge the new DataFrame with the original DataFrame based on week number and year
df_merged = df_all_dates.merge(df, on=['Week', 'Year'], how='left')

# Drop duplicate weeks, keeping the first occurrence
df_merged.drop_duplicates(subset=['Week', 'Year'], inplace=True)

df_merged.head()

Now we are going to convert types, this makes interpolation easier

In [ ]:
# Convert extension types to standard types for interpolation
for col in df_merged.select_dtypes(include=['category', 'Int64', 'UInt32', 'boolean']).columns:
    df_merged[col] = df_merged[col].astype('float64')
numeric_cols = df_merged.select_dtypes(include=[np.number])

numeric_cols.head()

Next up is the actual interpolation

In [ ]:
# Interpolate missing values for numeric columns
df_merged[numeric_cols.columns] = numeric_cols.interpolate(method='linear')

df_merged.head()

We can see that the values are filled in. Now we need to merge it on the analyst consensus and stock data

In [ ]:
# Merging the two DataFrames on 'Year' and 'Week' using an inner join.
final_df = pd.merge(df_merged, merged_df, on=['Year', 'Week'], how='inner')

# This will result in a DataFrame that only contains rows where both 'Year' and 'Week' match in both DataFrames.
# All weeks that do not exist in AnalystConsensus will be dropped.

# Displaying the shape of the new df
print(f"Merged DataFrame shape: {final_df.shape}")


final_df.head(30)

Now that everything is in one dataframe we will check the datatypes. We know that there might be problems with the asr financial data

In [ ]:
final_df.info()

We can see here that all datatype are correct but there is a dubplication of the date. We will remove this

In [ ]:
# Drop the 'date_y' column
final_df = final_df.drop(columns=['date_y'])
final_df = final_df.drop(columns=['Date_y'])
final_df = final_df.drop(columns=['Date_x'])

# Rename 'date_x' to 'date'
final_df = final_df.rename(columns={'date_x': 'date'})

# Now we check the dataframe
final_df.info()

# Phase 3

## Preprocessing
We will collect all imports at the top of this section to keep it clear what we are using

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = final_df

First we will look at the data we are working with using the .describe

In [ ]:
df.describe()

We can see that the target variable is heavily influenced to the buy side. There are no values for ``Underperform`` or ``Sell``. This is something to keep in mind for the modelling.

From Phase 2 we know that the total amount of the target variable switches every so often. It would be best to change these numbers to percentages based on the total count of the week. We will be doing this in the code block below step by step

In [ ]:
# Calculate the total number of ratings for each row
df['Total_Ratings'] = df[['Buy', 'Outperform', 'Hold', 'Underperform', 'Sell']].sum(axis=1)

# Convert each rating to a percentage of the total
for column in ['Buy', 'Outperform', 'Hold', 'Underperform', 'Sell']:
    df[column + '_%'] = (df[column] / df['Total_Ratings']) * 100

# Drop the original count columns and the total ratings column if they are no longer needed
df.drop(['Buy', 'Outperform', 'Hold', 'Underperform', 'Sell', 'Total_Ratings'], axis=1, inplace=True)

# Display the first few rows to verify the changes
df.head(30)

In the next step we will create a heatmap

In [ ]:
# Select only numeric columns for correlation calculation
numeric_df = df.select_dtypes(include=[np.number])

# Compute the correlation matrix
corr_matrix = numeric_df.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 8))

# Draw the heatmap
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')

# Title for the heatmap
plt.title('Heatmap of Correlation Matrix')

# Show the plot
plt.show()

As we have seen before there are 0 readings for ``Underperform_%`` or ``Sell_%``. This means our data is biased to the buy side.

Besides this we can see that there is a high correlation between every feature. This is good and strange, for now we will not investigate this due to time contraints

From the data of the stck market we will only take the `Open` and `Adj Close`. The other values from the stock data are very closely related but we want to keep the model simple to use and in this case more features that tell kind of the same will not make a big difference in performance

### Feature selection
From the correlation matrix above we can see that the `Open`, `High`, `Low`, `Close` and `Adj Close` are highly correlated. We will pick the `Close` feature .
It seems that the ``Volume`` does not have a high correlation but we will keep it in there for iteration zero.

It also seems that the ``Year`` has a high correlation to the target. Since we only have 1 full year we will not be using this.

In [42]:
# Define your target columns as a list of strings
target_columns = ['Buy_%', 'Outperform_%', 'Hold_%', 'Underperform_%', 'Sell_%']

# Include these target columns in your target DataFrame
target = df[target_columns]

# Define columns to exclude (including the target columns)
columns_to_exclude = target_columns + ['Week', 'Year', 'High', 'Low', 'Close']

# Select columns except the ones listed in columns_to_exclude
selected_columns = [col for col in numeric_df.columns if col not in columns_to_exclude]

# Create a DataFrame with only the selected columns
features = numeric_df[selected_columns]

# Define your features (X) and target (y)
X = features
y = target

### Splitting in to train/test
Commented due to different cross validation approach<br>
*The data we use is time series we need to keep this in mind with splitting the data. For now we will set the split at 80%.*

In [43]:
# # Set splitting point at 80%
# split_point = int(len(df) * 0.80)

# # Split the features and target into training and testing sets
# X_train, X_test = X[:split_point], X[split_point:]
# y_train, y_test = y[:split_point], y[split_point:]

# # Print the sizes of the train and test sets
# print(f"Training set size: {X_train.shape[0]} rows")
# print(f"Test set size: {X_test.shape[0]} rows")

## Modelling
In iteration zero we selected a Random Forest Regressor as our baseline model to predict weekly analyst consensus ratings, which are represented as percentages. Random Forest is chosen for its ability to handle complex and varied data types effectively. It's known for its robustness, reducing overfitting by averaging multiple decision trees.

For this iteration we will still be using the random forest regressor but we will take a different approach to our cross validation technique due to our very small dataset and keeping in mind that the data is time series.

We will be using Time-series cross validation, rolling cross-validation. This means we will comment out the original train/test split.

We will start with finding the optimal hyperparameters on the entire dataset. This is especially handy in our sitation of a small dataset.

In [ ]:
# Define a grid of hyperparameters to search over
param_grid = {
    'estimator__n_estimators': [100, 200, 300],
    'estimator__max_depth': [None, 10, 20, 30],
}

# Initialize the Random Forest model
random_forest_regressor = RandomForestRegressor(random_state=42)

# Wrap the model with MultiOutputRegressor
multi_target_regressor = MultiOutputRegressor(random_forest_regressor)

# Set up the grid search with cross-validation
grid_search = GridSearchCV(multi_target_regressor, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Perform the grid search on the full dataset
grid_search.fit(X, y)

# Retrieve the best model from the grid search
best_model = grid_search.best_estimator_

print(f"Best Hyperparameters: {grid_search.best_params_}")

Now we will start applying the rolling cross validation to the best model

In [ ]:
# Initialize lists to store the metrics for each roll
mse_scores = []
mae_scores = []
all_predictions = []
all_actuals = []
time_indices = []

initial_train_size = int(len(X) * 0.15)
roll_size = 1  # Set to 1 for a single-step roll forward

for i in range(initial_train_size, len(X) - roll_size):
    # Define the train and test sets for the current roll
    X_train, X_test = X[:i], X[i:i + roll_size]
    y_train, y_test = y[:i], y[i:i + roll_size]

    # Fit the best model on the current training set
    best_model.fit(X_train, y_train)

    # Make predictions on the current test set
    y_pred = best_model.predict(X_test)

    # Calculate and store the metrics for the current roll
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse_scores.append(mse)
    mae_scores.append(mae)

    # Store predictions and actual values
    all_predictions.append(y_pred[0])  # Assuming y_pred is a 1D array
    all_actuals.append(y_test.iloc[0])  # Assuming y_test is a DataFrame
    time_indices.append(i)  # Or your specific time index

# Calculate the average of the metrics across all rolls
mean_mse = np.mean(mse_scores)
mean_mae = np.mean(mae_scores)

print(f"Rolling Cross-Validated Mean Squared Error: {mean_mse}")
print(f"Rolling Cross-Validated Mean Absolute Error: {mean_mae}")

## Evaluation

With this approach we can see that the Mean Squared Error and Mean Absolute Error both improved.

**Old** Result standard cross validation
- Mean Squared Error: 11.034635994042576
- Mean Absolute Error: 2.2646886446886447
- Cross-validated MSE: 5.128838594372667

**New** Results rolling cross validation
- Rolling Cross-Validated Mean Squared Error: 10.791339472554373
- Rolling Cross-Validated Mean Absolute Error: 1.6185091575091584

The new result look better then the previous approach. With the initial train size set to .80 there is a change that we are overfitting.

In [46]:
# Convert lists to arrays for easier handling
all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

In [ ]:
# Visualization
plt.figure(figsize=(16, 12))

# Assuming you have a list of target names
target_names = ['Buy', 'outperform', 'Hold', 'Underperform', 'Sell']
colors = ['blue', 'green', 'red', 'purple', 'orange']

# Actual vs. Predicted values
plt.subplot(2, 1, 1)
for i, target_name in enumerate(target_names):
    actual_color = colors[i % len(colors)]
    predicted_color = 'grey'
    plt.plot(time_indices, all_actuals[:, i], label=f'Actual - {target_name}', color=actual_color, marker='o')
    plt.plot(time_indices, all_predictions[:, i], label=f'Predicted - {target_name}', color=predicted_color, linestyle='--', marker='x')
plt.title('Actual vs. Predicted Values')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()

# Residuals
plt.subplot(2, 1, 2)
for i, target_name in enumerate(target_names):
    residuals = all_actuals[:, i] - all_predictions[:, i]
    plt.plot(time_indices, residuals, label=f'Residuals - {target_name}', color=colors[i % len(colors)], marker='o')
plt.axhline(y=0, color='black', linestyle='--')
plt.title('Residuals Over Time')
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.legend()

plt.tight_layout()
plt.show()

## Plot Descriptions

### Actual vs. Predicted Values

The top plot displays the actual and predicted values over time/iterations for different categories: Buy, Outperform, Hold, Underperform, and Sell. Each category is represented by a unique color with actual values depicted by solid lines and predicted values by dashed lines with markers. This plot allows us to compare the model's predictions against the real-world data across various investment decisions over a period of time.
We can see that the line for Underperform and Sell stays at the same height. This is due to there being no values for these targets in our current dataset

### Residuals Over Time

The bottom plot shows the residuals over time for the same categories. Residuals are the differences between actual and predicted values, indicating the error of the predictions. Consistent with the top plot, each category has a unique color. A residual value of zero would mean a perfect prediction, while any deviation from zero represents an error.When the residuals are small its better. We can see that after 4 iterations the residuals increase but after 9 iterations they all are returning to the 0 line

### Conclusion
We think that it is not possible to optimize the model further with the current dataset.  


In a next iteration we need to work with more data to get a better model. With the limited data this is very hard to achieve. 

# Phase 4
## Demonstration
In the following section we will create the code that can be used to make a prediction for our stakeholder. After we check if it works we are exporting the model and dataframes necessary

In [ ]:
features.head(0)

In [49]:
def predict_analyst_category(model, features, target_names):
    """
    Predict the analyst category ('Buy', 'Hold', or 'Sell') based on;
    'Return_On_Equity(%)', 'OCI', 'Combined_ratio_P&C_and_Disabilty(%)', 'Outstanding_shares(weighted_average)', 
    'Open', 'Adj Close', 'Volume' using the trained model.

    :param model: Trained MultiOutputRegressor model.
    :param features: dataframe that holds all features used for training the model
    :param target_names: List of target variable names in the order they were used during model training.
    :param feature_names: List of feature names as they were used during model training.
    :return: Predicted category.
    """
    # Create a DataFrame for the input features with the correct column names
    input_df = features

    # Predict using the model
    predictions = model.predict(input_df)[0]

    # Map predictions to their corresponding target names
    prediction_dict = dict(zip(target_names, predictions))

    # Aggregate predictions
    buy_prediction = prediction_dict['Buy_%'] + prediction_dict['Outperform_%']
    sell_prediction = prediction_dict['Sell_%'] + prediction_dict['Underperform_%']
    hold_prediction = prediction_dict['Hold_%']

    categories = {'Buy': buy_prediction, 'Hold': hold_prediction, 'Sell': sell_prediction}
    predicted_category = max(categories, key=categories.get)

    return predicted_category

In [ ]:
Return_On_Equity = 10
OCI	= 10
Combined_ratio_PC_and_Disabilty = 10
Outstanding_shares_weighted_average = 10
Open = 10
Adj_Close = 10
Volume = 10


values = [Return_On_Equity, OCI, Combined_ratio_PC_and_Disabilty, Outstanding_shares_weighted_average, Open, Adj_Close, Volume]
input_data = pd.DataFrame([values], columns=features.columns)


target_names = ['Buy_%', 'Outperform_%', 'Hold_%', 'Underperform_%', 'Sell_%']
predicted_category = predict_analyst_category(best_model, input_data, target_names)
print("Predicted Analyst Consensus:", predicted_category)

### Export model for flask app

In [ ]:
from joblib import dump

dump(best_model, 'model.joblib')

In [ ]:
# Create an empty DataFrame with the same structure as X_train
df_structure = pd.DataFrame({col: pd.Series(dtype=typ) for col, typ in X_train.dtypes.items()})

# Save the structure
df_structure.to_pickle('df_structure.pkl')

df_structure.head()

## Feedback

See file named demonstration_plan

# Conclusion

## Iteration 1
*Date: 25/01/2024*<br>
During this iteration it become clear that with the limited availability in history of the analyst consensus and limited availability of recent data from asr due to a new standard named IFRS 17 / 9 it is not possible to drastically improve the model. 

### IFRS 17 / 9
This new standard makes it that there is better transparency in the insurance sector. The problem is that this new standard affects all important features that we need according to our domain understanding. The new regulations came in to effect on Januari 2023. Due to this we must choose to collect data after this date or before.

#### After
With the collection after Januari 2023 we only have one row of data. Asr only published one Half Year report. The Full Year report for 2023 is expected around May 2024

#### Before
When we look before Januari 2023 we can go back as far as 2016, on this date asr went public. The problem with collecting information before januari 2023 is the data for the Analyst consensus.

### Analyst consensus
The anaylyst consensus is currently collected from a website with the url marketwatcher.com. They offer the longest history of 18 months back with a moving time frame. That means the 18 months back is always from the current date. This means the data collection can not be earlier then June 2022. There are no other free soruces available that offer a longer history.
Because of this we can not use the data from asr before 2022.

### Bias with current data
Another problem is the current bias in the data. Currently there are no underperform or sell values in our dataset. This makes it that the model can not predict these values and will always give the advice to either buy or hold.

### Possible solution
#### Multiple insurance companies
To continue with a next sprint we would need more data, from asr we can not get more data. We could only look at different companies in the same sector like NN, Achmea, or Allianz. When we collect the same data from these companies then there is more data to work with.

#### Paid data
Another option would be to pay for the data, this would solve most problems. Anotrher upside would also be that data can be more easily collected and this would speed up each following iteration.


## Older Iterations, Notebook ends here
### Iteration Zero

#### Data Limitations
Our analysis in phase 2 faced challenges due to recent changes in financial calculations, effective from the start of 2023. These changes limited our ability to utilize our initially identified data effectively. Furthermore, the restriction in accessing historical analyst consensus data, available only for the past 18 months, resulted in a data availability mismatch. This limitation prevented us from leveraging older, potentially valuable data sets.

#### Model Performance
The baseline model underperformed, which could be attributed to insufficient optimization of the target variables and the limited size of our dataset. Additionally, the dataset lacked some key features that might be crucial in predicting the analyst consensus.

#### Future Iterations
Looking ahead, we need to find ways to circumvent the financial calculation changes or access more extensive historical data for the analyst consensus. Enhancing our target variable optimization and refining the model to predict these targets more accurately will be a focus. Addressing potential biases and improving model performance may become feasible with a more comprehensive dataset.